In [3]:
#  Change detect type
# input: nPosts

import difflib
import sys
import re
import nltk

In [164]:
d1 = '\
# Section0\n\
<del>A example sentence that is embedded. The embedding is compared withanother</del><ins>Faselbla</ins>\n\
# Section1'

d2 = '\
# Section0\n\
Faselbla\n\
# Section1\n\
<ins>wort wort wort wort</ins>\n'

d3 = '\
# Section0\n\
Faselbla\n\
# Section1\n\
wort wort wort wort<ins>wort wort</ins>\n'

d4 = '\
# Section0<ins>adas</ins>\n\
Faselbla\n\
# Section1\n\
wort wort wort wort wort wort'

d5 = '\
# Section0<ins>wort</ins>\n\
Faselbla\n\
# Section1\n\
wort wort wort wort wort wort'

d6 = '\
# Section0<ins>wort wort wort wort</ins>\n\
Faselbla\n\
# Section1\n\
wort wort wort wort wort wort'

d = [d1, d2, d3, d4, d5, d6]

In [165]:
def count_changes(difference):
    # Takes difference from wdiffhtml as input
    # Returns total number of changes in words
    dif = difference.splitlines()
    num_changes = 0
    for line in dif:
        if '<ins>' in line:
            word_list = nltk.word_tokenize(
                line[re.search('<ins>', line).end():re.search('</ins>', line).start()])
            num_changes += len(word_list)
        if '<del>' in line:
            word_list = nltk.word_tokenize(
                line[re.search('<del>', line).end():re.search('</del>', line).start()])
            num_changes += len(word_list)
    return num_changes

In [166]:
def which_paragraph_changed(text):
    list_of_paragraphs = re.findall(r"[^#]+", text)
    has_changes = [_check_if_has_diff(paragraph) for paragraph in list_of_paragraphs]
    #print(has_changes)
    return has_changes

def _check_if_has_diff(text):
    diff_tokens = ["<ins>", "<del>"]
    return any(diff_token in text for diff_token in diff_tokens)

In [167]:
paragraph_treshold = 5  # at least this many word changes have to be added/deleted in a paragraph to qualify for a paragraph commit
# POSTs without changes are discarded (assumption) so I assume every POST here has actual changes

scope_hist = []  # record scopes aka edited paragraphs of a POST
count_hist = []  # record count of words added/deleted by a POST
posts_waiting = False  # are posts queued for a commit?
scope_switched = False  # did the scope switch since the last POST?
for i, difference in enumerate(d):
    # Check scopes
    changes = which_paragraph_changed(difference)
    scope = np.arange(len(changes))[changes] if changes is not None else []
    scope_hist.append(scope)
    single_scope = len(scope == 1)  # only POSTs that change a single Scope can be "§ edit" commits
    if len(scope_hist) > 1:
        scope_switched = (scope==scope_hist[-1])
        posts_waiting = True
    else:
        scope_switched = False
    
    count_hist.append(count_changes(difference))
    
    # Check if § was added TODO
    num_of_paragraphs = len(changes)
    
    # Do commit
    # TODO COMMIT and delete corresponding histories
    # find out which POSTs to commit
    if single_scope:
        current_scope = scope[0]
        words_changed = 0
        for i in reversed(range(len(scope_hist))):
            if scope_hist[i] != current_scope:
                break
            else:
                words_changed += count_hist[i]
        print('POST added ',words_changed, ' words in §', scope[0])
    
    if posts_waiting and words_changed >= paragraph_treshold: # then we made enough changes to qualify for a paragraph commit
        print('You should commit POST ', i, ' as:')
        if np.unique(scope_hist[:i]) > 1:
            print('small changes')
        else:
            print('Edit of § ', scope_hist[0][0])
        posts_waiting = False
        del(scope_hist[:i])
        del(count_hist[:i])


POST added  13  words in § 0
POST added  4  words in § 1
POST added  6  words in § 1
You should commit POST  0  as:
Edit of §  0
POST added  1  words in § 0
POST added  2  words in § 0
POST added  6  words in § 0
You should commit POST  2  as:


C:\Users\Stefan\Miniconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:40: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()